In [13]:
import numpy as np
from numpy.linalg import eig
import matplotlib
import math
import tkinter as tk
from tkinter import ttk
np.set_printoptions(suppress=True)
matplotlib.use('TkAgg')
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg,
    NavigationToolbar2Tk
)
from PIL import Image,ImageTk
#####先創造主體並固定大小以免排版跑掉#####
class PopulationApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.title('族群預估')
        self.geometry('1280x720')
        self.resizable(0, 0)
        self._frame = None
        self.switch_frame(Homepage)
        
####透過switch frame函示來跳轉到不同功能的頁面##
    def switch_frame(self, frame_class):
        new_frame = frame_class(self)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.place(relx=0,rely=0)
####主頁面，介面進入點會有兩個按鈕引導使用者到不同功能的頁面##        
class Homepage(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        tk.Frame.configure(self,width=1280,height=720)
        img=Image.open("pokemon2.jpg")
        img1=img.resize((1280,640))
        photo1=ImageTk.PhotoImage(img1)
        label2=tk.Label(self,image=photo1)
        label2.image=photo1
        label2.place(x=0,y=0)
####主要有兩個功能##        
        label1=tk.Label(self, text="動物族群預測程式", font=('Helvetica', 25, "bold")).place(relx=0.386,rely=0.9)
        tk.Button(self, text="族群結構預測",font=('Helvetica', 25, "bold"),bg='#99FF4D',
                  command=lambda: master.switch_frame(Lesile)).place(relx=0.25,rely=0.8)
        tk.Button(self, text="族群遷徙預測",font=('Helvetica', 25, "bold"),bg='#99FF4D',
                  command=lambda: master.switch_frame(Markov)).place(relx=0.565,rely=0.8)
        
        
###功能一:族群年齡購預測的函式###       
class Lesile(tk.Frame):
    def __init__(self,master):
       tk.Frame.__init__(self, master)
       tk.Frame.configure(self,bg='#99FF4D',width=1280,height=180)
       label1=tk.Label(self,text='請輸入幼牛數量(0~2歲)  ',font=("微軟正黑體", 15),bg='#7CFC00')
       label1.place(relx=0.1,rely=0.1)
       self.entry1=tk.Entry(self,font=("微軟正黑體", 15))
       self.entry1.place(relx=0.35,rely=0.1)
       label2=tk.Label(self,text='請輸入成年牛數量(3~4歲)',font=("微軟正黑體", 15),bg='#7CFC00')
       label2.place(relx=0.1,rely=0.3)
       self.entry2=tk.Entry(self,font=("微軟正黑體", 15))
       self.entry2.place(relx=0.35,rely=0.3)
       label3=tk.Label(self,text='請輸入老年牛數量(5~6歲)',font=("微軟正黑體", 15),bg='#7CFC00')
       label3.place(relx=0.1,rely=0.5)
       self.entry3=tk.Entry(self,font=("微軟正黑體", 15))
       self.entry3.place(relx=0.35,rely=0.5)
####透過三個按鈕來操作功能送出按鈕會正式開始計算並繪圖，清除則是清除輸入區，返回則是返回封面頁##
       button2=tk.Button(self,text='清除',command=self.clear,font=("微軟正黑體", 10))
       button2.place(relx=0.75,rely=0.3)
####送出按鈕會引導到submit函式###
       button1=tk.Button(self,text='送出',command=self.submit,font=("微軟正黑體",10))
       button1.place(relx=0.75,rely=0.1)
       button3=tk.Button(self,text='返回',command=lambda: master.switch_frame(Homepage),font=("微軟正黑體",10))
       button3.place(relx=0.75,rely=0.5)
       self.frame1=tk.Frame(root,width=360,height=180,bg='green')
       self.frame1.place(relx=0.72,rely=0.25)
       
       frame2=tk.Frame(root,width=360,height=360)
       frame2.place(relx=0.72,rely=0.5)
       img=Image.open("cow.jpg")
       img1=img.resize((360,360))
       photo1=ImageTk.PhotoImage(img1)
       label0=tk.Label(frame2,image=photo1)
       label0.image=photo1
       label0.place(x=0,y=0)
###clear function清除輸入區，和清除按鈕式有連接的##       
    def clear(self):
       self.entry1.delete(0,'end')
       self.entry2.delete(0,'end')
       self.entry3.delete(0,'end')
##submit 按鈕會把輸入區的字串讀進來，並傳入cowPredict函式計算
    def submit(self):    
        if not(str.isdigit(self.entry1.get())) or not(str.isdigit(self.entry2.get())) or not(str.isdigit(self.entry3.get())):
           tk.messagebox.showerror('message', '請輸入大於或等於0的整數')
        else:
           x=self.entry1.get()
           y=self.entry2.get()
           z=self.entry3.get()
           self.cowPredict(x,y,z)
##cowPredict function 中 powermethod會使用到
    def normalize(self,x):
        fac = max(abs(x))
        x_n = x / max(x)
        return fac, x_n
##cowPredict函式使用會建構lesile矩陣進行計算並使用matplot繪圖後以canvas呈現在視窗上    
    def cowPredict(self,x,y,z):
        f=Figure(figsize=(8,5))    
        plt=f.add_axes([0.1,0.1,0.8,0.8])
        plt.cla()
##資料預處理鄉輸入區字串轉成整數        
        x=int(x)
        y=int(y)
        z=int(z)
##lesile 矩陣假設成牛兩年可以生3隻小牛，老年牛則是2隻，小牛生存率為0.8、成年牛0.7##
        A=np.array([[0,3,1],[0.8,0,0],[0,0.7,0]])
##years表示預估多少個單位時間(一單位2年)
        years=3
        row=3
        col=years+1
##用3個list記錄各年齡不同時間的數量
        young=[]
        adult=[]
        elderly=[]
##族群向量predict儲存各時期的族群結構，並把輸入資料加入向量中的第0欄
        predict=np.zeros((col,row))
        now=np.array([x,y,z])
        predict[0]=np.array([x,y,z])
        k=now
##接著進行3次lesile矩陣乘法，每次都會把預測結果加入predict矩陣
        year=np.arange(years+1)
        for i in range(1,years+1):
            k=np.ceil(np.dot(A,k))    
            predict[i]=k
##把各時間不同年齡的數量加入list中，為繪圖做準備
        for i in range(years+1):
            young.append(predict[i][0])
            adult.append(predict[i][1])
            elderly.append(predict[i][2])
##name為x軸標籤
        name=['now','2024','2026','2028','2030','2032']  
##接著是用power method求lesile矩陣的穩定狀態下族群的結構會是如何
##即找出最大的eigenvalue對應的eigenvector
        x1=np.array([1,1,1])
        for i in range(100):
                x1 = np.dot(A, x1)
                lambda_1, x1 =self.normalize(x1)
##輸出穩定結構
        if(x1.all()>0):
            label3=tk.Label(self.frame1,text='穩定之結構比例為',font=("微軟正黑體", 15))
            label3.place(relx=0.1,rely=0.2)
            x1=x1/min(x1)
            x1=np.around(x1,decimals=0)
            x1=np.array2string(x1)
            label4=tk.Label(self.frame1,text=x1,font=("微軟正黑體", 15))
            label4.place(relx=0.1,rely=0.4)
        else:
            label3.forget()
##繪製堆積長條圖呈現每2年各年齡結構的變化        
        plt.bar(year,young,color='blue',label='infant')
        plt.bar(year,adult,color='green',bottom=young,label='adult')
        plt.bar(year,elderly,color='red',bottom=np.array(adult)+np.array(young),label='elderly')
        plt.set_xticks(year)
        plt.set_xticklabels(name[0:years+1])
        plt.set_xlabel('Years')
        plt.set_ylabel('Cow Population')
        plt.set_title('Cow Population Predict')
        plt.legend()
        canvas = FigureCanvasTkAgg(f,master=root)
        canvas.draw()
        canvas.get_tk_widget().place(relx=0.05,rely=0.28)
##功能二動物遷徙預測使用馬可夫轉移矩陣實作，分為2x2和3x3的版本        
class Markov(tk.Frame):
    def __init__(self,master):
       tk.Frame.__init__(self, master)
       tk.Frame.configure(self,width=1280,height=720)
       img=Image.open("pokemon3.jpg")
       img1=img.resize((1280,720))
       photo1=ImageTk.PhotoImage(img1)
       label=tk.Label(self,image=photo1)
       label.image=photo1
       label.place(x=0,y=0)
##在frame下創造兩個分頁方便使用者切換
       notebook=ttk.Notebook(self)
       notebook.place(relx=0.1,rely=0.05)
       self.twoMatrix = tk.Frame(notebook,width=1024,height=630,bg='#F0F8FF')
       self.threeMatrix = tk.Frame(notebook,width=1024,height=630,bg='#F0F8FF')
       notebook.add(self.twoMatrix, text="2地遷徙") 
       notebook.add(self.threeMatrix, text="3地遷徙")
       
       ##2x2 Markov matrix的介面
       label1=tk.Label(self.twoMatrix,text="請輸入地點1名稱:",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.05)
       self.entry1=tk.Entry(self.twoMatrix,font=("微軟正黑體", 12))
       self.entry1.place(relx=0.21,rely=0.06)
       label2=tk.Label(self.twoMatrix,text="請輸入地點2名稱:",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.1)
       self.entry2=tk.Entry(self.twoMatrix,font=("微軟正黑體", 12))
       self.entry2.place(relx=0.21,rely=0.11)
       label3=tk.Label(self.twoMatrix,text="請輸入地點1往地點2之遷徙率(0~1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.17)
       self.entry3=tk.Entry(self.twoMatrix,font=("微軟正黑體", 12),width=5)
       self.entry3.place(relx=0.38,rely=0.18)
       label4=tk.Label(self.twoMatrix,text="請輸入地點2往地點1之遷徙率(0~1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.22)
       self.entry4=tk.Entry(self.twoMatrix,font=("微軟正黑體", 12),width=5)
       self.entry4.place(relx=0.38,rely=0.23)
       label5=tk.Label(self.twoMatrix,text="請輸入今年地點1的動物比例:(0~1)",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.27)
       self.entry5=tk.Entry(self.twoMatrix,font=("微軟正黑體", 12),width=5)
       self.entry5.place(relx=0.38,rely=0.28)
       label6=tk.Label(self.twoMatrix,text="請輸入今年地點2的動物比例:(0~1)",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.32)
       self.entry6=tk.Entry(self.twoMatrix,font=("微軟正黑體", 12),width=5)
       self.entry6.place(relx=0.38,rely=0.33)
##同樣使用按鈕連結到submit和clear函式
       self.text1=tk.StringVar()
       button2=tk.Button(self.twoMatrix,text='清除',command=self.clear,font=("微軟正黑體", 10))
       button2.place(relx=0.5,rely=0.15)
       button1=tk.Button(self.twoMatrix,text='送出',command=self.submit,font=("微軟正黑體",10))
       button1.place(relx=0.5,rely=0.08)
       button3=tk.Button(self.twoMatrix,text='返回',command=lambda: master.switch_frame(Homepage),font=("微軟正黑體",10))
       button3.place(relx=0.5,rely=0.22)
       label21=tk.Label(self.twoMatrix,text="*每地區各自遷出請勿超過1",font=("微軟正黑體", 15),fg='red',bg='#F0F8FF').place(relx=0.6,rely=0.12)
       label22=tk.Label(self.twoMatrix,text="*每一欄位都不包含0或1兩地族群總和須為1",font=("微軟正黑體", 15),fg='red',bg='#F0F8FF').place(relx=0.6,rely=0.17)
       label23=tk.Label(self.twoMatrix,text="*兩地族群總和須為1",font=("微軟正黑體", 15),fg='red',bg='#F0F8FF').place(relx=0.6,rely=0.21) 
       self.labels=tk.Label(self.twoMatrix,textvariable=self.text1,font=("微軟正黑體",15),bg='#F0F8FF').place(relx=0.7,rely=0.45) 
       #3x3介面
       label7=tk.Label(self.threeMatrix,text="請輸入地點1:",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.05)
       self.entry7=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=10)
       self.entry7.place(relx=0.2,rely=0.05)
       label8=tk.Label(self.threeMatrix,text="請輸入地點2:",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.35,rely=0.05)
       self.entry8=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=10)
       self.entry8.place(relx=0.5,rely=0.05)
       label9=tk.Label(self.threeMatrix,text="請輸入地點3:",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.65,rely=0.05)
       self.entry9=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=10)
       self.entry9.place(relx=0.8,rely=0.05)
       label16=tk.Label(self.threeMatrix,text="今年地點1的動物比例(<1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.11)
       self.entry16=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=5)
       self.entry16.place(relx=0.29,rely=0.11)
       label17=tk.Label(self.threeMatrix,text="今年地點2的動物比例(<1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.35,rely=0.11)
       self.entry17=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=5)
       self.entry17.place(relx=0.59,rely=0.11)
       label18=tk.Label(self.threeMatrix,text="今年地點3的動物比例(<1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.65,rely=0.11)
       self.entry18=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=5)
       self.entry18.place(relx=0.89,rely=0.11)
       label10=tk.Label(self.threeMatrix,text="請輸入地點1往地點2之遷徙率(0~1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.17)
       self.entry10=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=5)
       self.entry10.place(relx=0.4,rely=0.17)
       label11=tk.Label(self.threeMatrix,text="請輸入地點1往地點3之遷徙率(0~1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.5,rely=0.17)
       self.entry11=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=5)
       self.entry11.place(relx=0.84,rely=0.17)
       label12=tk.Label(self.threeMatrix,text="請輸入地點2往地點1之遷徙率(0~1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.23)
       self.entry12=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=5)
       self.entry12.place(relx=0.4,rely=0.23)
       label13=tk.Label(self.threeMatrix,text="請輸入地點2往地點3之遷徙率(0~1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.5,rely=0.23)
       self.entry13=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=5)
       self.entry13.place(relx=0.84,rely=0.23)
       label14=tk.Label(self.threeMatrix,text="請輸入地點3往地點1之遷徙率(0~1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.05,rely=0.29)
       self.entry14=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=5)
       self.entry14.place(relx=0.4,rely=0.29)
       label15=tk.Label(self.threeMatrix,text="請輸入地點3往地點2之遷徙率(0~1):",font=("微軟正黑體", 15),bg='#F0F8FF').place(relx=0.5,rely=0.29)
       self.entry15=tk.Entry(self.threeMatrix,font=("微軟正黑體", 12),width=5)
       self.entry15.place(relx=0.84,rely=0.29)
       label19=tk.Label(self.threeMatrix,text="*每地區各自遷出總和請勿超過1",font=("微軟正黑體", 15),fg='red',bg='#F0F8FF').place(relx=0.05,rely=0.35)
       label20=tk.Label(self.threeMatrix,text="*每一欄位都不包含0或1，且三地族群總和須為1",font=("微軟正黑體", 15),fg='red',bg='#F0F8FF').place(relx=0.5,rely=0.35)
##按鈕連結計算函式
       button4=tk.Button(self.threeMatrix,text='清除',command=self.clearr,font=("微軟正黑體", 10))
       button4.place(relx=0.9,rely=0.17)
       button5=tk.Button(self.threeMatrix,text='送出',command=self.submit1,font=("微軟正黑體",10))
       button5.place(relx=0.9,rely=0.23)
       button6=tk.Button(self.threeMatrix,text='返回',command=lambda: master.switch_frame(Homepage),font=("微軟正黑體",10))
       button6.place(relx=0.9,rely=0.29)
##用來清除2x2矩陣輸入區的function       
    def clear(self):
       self.entry1.delete(0,'end')
       self.entry2.delete(0,'end')
       self.entry3.delete(0,'end')    
       self.entry4.delete(0,'end')
       self.entry5.delete(0,'end')    
       self.entry6.delete(0,'end')
##submit函式從2x2介面中讀取動物在各棲息地分布的數量，以及他們往外遷移的比率    
    def submit(self):
        A=self.entry1.get()
        B=self.entry2.get()
        if(A=="" or B==""):
          tk.messagebox.showerror('message','請輸入地名')
        else:
##x,y為族群遷徙比率，因為只有兩個棲息地，所以每個地方只需要遷出的資料，遷留在原地的動物數就用1-x或1-y可以得到
##i、j為兩棲息地的動物數量占整個族群的比率，相加=1
            x=self.entry3.get()
            y=self.entry4.get()
            i=self.entry5.get()
            j=self.entry6.get()
            k=np.zeros(4)
##資料預處理為了處理方便我們捨去一個棲息地全部遷出的(x=1,y=1)情況，因此只需檢查輸入是否為合法小數並過濾字串
            if not(str.isdigit(x)) and x.find('.')!=-1 and not(str.isdigit(y)) and y.find('.')!=-1 and not(str.isdigit(i)) and i.find('.')!=-1 and not(str.isdigit(j)) and j.find('.')!=-1:
               a,b=self.entry3.get().split('.')
               c,d=self.entry4.get().split('.')
               e,f=self.entry5.get().split('.')
               g,h=self.entry6.get().split('.')
               if not(str.isdigit(a)) or not(str.isdigit(b)) or not(str.isdigit(c)) or not(str.isdigit(d)) or not(str.isdigit(e)) or not(str.isdigit(f)) or not(str.isdigit(g)) or not(str.isdigit(h)):
                   tk.messagebox.showerror('message','遷徙率及比例請輸入介於0~1之間的小數')
               ##避免遷出比率為1
               elif float(x)>1 or float(y)>1 or float(i)>1 or float(j)>1:
                   tk.messagebox.showerror('message','遷徙率及比例請輸入介於0~1之間的小數')
              ##族群總數必須等於1
               elif(float(i)+float(j)>1.00001):
                   tk.messagebox.showerror('message','族群總和需為1')
               ##確定輸入合法後再傳入cal function計算
               else:
                   k=np.array([float(x),float(y),float(i),float(j)])
                   self.cal(k)
            else:
                   tk.messagebox.showerror('message','遷徙率及比例請輸入介於0~1之間的小數')
##計算每年動物遷移量及棲息狀況    
    def cal(self,k):
            ##name list從輸入區儲存地名，first儲存棲息地1每次遷徙後的數量比率，second則儲存棲息地2
            name=[]
            first=[]
            second=[]
            x=[]
            f=Figure(figsize=(6,4))    
            plt=f.add_axes([0.15,0.2,0.7,0.7])
            plt.cla()
            name1=['now','2023','2024','2025','2026','2027']
            name.append(self.entry1.get())
            name.append(self.entry2.get())
            A=np.zeros((2,2))
##創造2x2馬可夫矩陣，且每行總和都必須為1
            A[0][0]=1-k[0]
            A[1][0]=k[0]
            A[0][1]=k[1]
            A[1][1]=1-k[1]
##x儲存每次遷徙前棲息地的動物比率
            for i in range(2,4):
                x.append(k[i])
##years表示預估5年
            years=5
            year=np.arange(years+1)
            k=np.array(x)
##將今年的情況加入first second中幫助繪圖
            first.append(100*k[0])
            second.append(100*k[1])
##進行5次馬可夫矩陣乘法，並把每次兩棲息地的數量比率放入
            for i in range(1,years+1):
                k=np.dot(A,k)
                first.append(100*k[0])
                second.append(100*k[1])
##接著繪製長條圖，來顯示各地區分布狀況
            width=0.4
            plt.bar(year-0.2,first,width,color='blue',label=name[0])
            plt.bar(year+0.2,second,width,color='green',label=name[1])
            plt.set_xticks(year)
            plt.set_xticklabels(name1[0:years+1])
            plt.set_xlabel('Years')
            plt.set_ylabel('Animals Population(%)')
            plt.set_title('Animals Population Predict')
            plt.legend()
            canvas = FigureCanvasTkAgg(f,master=self.twoMatrix)
            canvas.draw()
            canvas.get_tk_widget().place(relx=0.057,rely=0.4)
            
##求出馬可夫穩定狀態，由於2x2矩陣形式只有幾種可以容易判斷是否存在穩定
            label2=tk.Label(self.twoMatrix,text="兩地之數量趨於穩定時為",font=("微軟正黑體",15),bg='#F0F8FF').place(relx=0.7,rely=0.4)
    
##若存在穩定則把eigenvalue 1對穎到的eigenvector求出就是穩定狀態下的比率
            eVal,vector=eig(A)
            counter=0
            for i in eVal:
                if i==1  :
                    break
                counter+=1 
            flag=0
##並將穩定狀態乘上上總數求出數量
            stable=np.array([vector[0][counter],vector[1][counter]])
            stable=np.around(stable/min(stable),decimals=2) 
            self.text1.set(str(stable[0])+":"+str(stable[1]))
            
            

    ##3x3 馬可夫矩陣
##3x3版clear矩陣清除輸入區
    def clearr(self):
       self.entry7.delete(0,'end')
       self.entry8.delete(0,'end')
       self.entry9.delete(0,'end')    
       self.entry10.delete(0,'end')
       self.entry11.delete(0,'end')    
       self.entry12.delete(0,'end')
       self.entry13.delete(0,'end')
       self.entry14.delete(0,'end')    
       self.entry15.delete(0,'end')
       self.entry16.delete(0,'end')
       self.entry17.delete(0,'end')    
       self.entry18.delete(0,'end')
##subnit1 同subnit功能只是為3x3設計        
    def submit1(self):
        A=self.entry7.get()
        B=self.entry8.get()
        C=self.entry9.get()
        if(A=="" or B=="" or C==""):
          tk.messagebox.showerror('message','請輸入地名')
        else:
            x=self.entry10.get()
            y=self.entry11.get()
            i=self.entry12.get()
            j=self.entry13.get()
            e=self.entry14.get()
            f=self.entry15.get()
            g=self.entry16.get()
            h=self.entry17.get()
            z=self.entry18.get()
            k=np.zeros(9)
##過濾非小數輸入            
            if (not(str.isdigit(x)) and x.find('.')!=-1 and not(str.isdigit(y)) and y.find('.')!=-1 and not(str.isdigit(i)) and i.find('.')!=-1 and not(str.isdigit(j)) and j.find('.')!=-1 and
            not(str.isdigit(e)) and e.find('.')!=-1 and not(str.isdigit(f)) and f.find('.')!=-1 and not(str.isdigit(g)) and g.find('.')!=-1 and not(str.isdigit(h)) and h.find('.')!=-1 and not(str.isdigit(z)) and z.find('.')!=-1):
               a,b=self.entry10.get().split('.')
               c,d=self.entry11.get().split('.')
               e1,f1=self.entry12.get().split('.')
               g1,h1=self.entry13.get().split('.')
               a1,b1=self.entry14.get().split('.')
               c1,d1=self.entry15.get().split('.')
               e2,f2=self.entry16.get().split('.')
               g2,h2=self.entry17.get().split('.')
               a3,b3=self.entry18.get().split('.')
##3x3中遷徙率每個地區需要紀錄兩筆，滯留率則用1-前面兩筆就可以求出        ，同時檢查是否有非法輸入
               if (not(str.isdigit(a)) or not(str.isdigit(b)) or not(str.isdigit(c)) or not(str.isdigit(d)) or not(str.isdigit(e1)) or not(str.isdigit(f1)) or not(str.isdigit(g1)) or not(str.isdigit(h1))
                  or not(str.isdigit(a1)) or not(str.isdigit(b1)) or not(str.isdigit(c1)) or not(str.isdigit(d1)) or not(str.isdigit(e2)) or not(str.isdigit(f2)) or not(str.isdigit(g2)) or not(str.isdigit(h2))
                  or not(str.isdigit(a3)) or not(str.isdigit(b3))):
                   tk.messagebox.showerror('message','遷徙率及比例請輸入介於0~1之間的小數')
               elif float(x)>1 or float(y)>1 or float(i)>1 or float(j)>1 or float(e)>1 or float(f)>1 or float(g)>1 or float(h)>1 or float(z)>1:
                   tk.messagebox.showerror('message','遷徙率及比例請輸入介於0~1之間的小數')
               elif(float(x)+float(y)>1):
                   tk.messagebox.showerror('message','地點1之遷出率不超過1')
               elif(float(i)+float(j)>1):
                   tk.messagebox.showerror('message','地點2之遷出率不超過1')
               elif(float(e)+float(f)>1):
                   tk.messagebox.showerror('message','地點3之遷出率不超過1')
               elif((float(g)+float(h)+float(z))>1.00001):
                   tk.messagebox.showerror('message','族群的總和必須為1')
##確認輸入正確後傳到cal1 function執行
               else:
                   k=np.array([float(x),float(y),float(i),float(j),float(e),float(f),float(g),float(h),float(z)])
                   self.cal1(k)
            else:
                   tk.messagebox.showerror('message','遷徙率及比例請輸入介於0~1之間的小數')    
##3x3版cal function
    def cal1(self,k):
        ##name存地名，三個list存遷徙後的比率
        name=[]
        first=[]
        second=[]
        third=[]
        x=[]
        f=Figure(figsize=(6,4))    
        plt=f.add_axes([0.15,0.2,0.7,0.7])
        plt.cla()
        name1=['now','2023','2024','2025','2026','2027']
##從輸入區讀取地名
        name.append(self.entry7.get())
        name.append(self.entry8.get())
        name.append(self.entry9.get())
        A=np.zeros((3,3)) 
##建構3x3馬可夫矩陣            
        A[0][0]=1-k[0]-k[1]
        A[1][0]=k[0]
        A[2][0]=k[1]
        A[0][1]=k[2]
        A[1][1]=1-k[2]-k[3]
        A[2][1]=k[3]
        A[0][2]=k[4]
        A[1][2]=1-k[4]-k[5]
        A[2][2]=k[5]
        
##把現在的分布情形加入
        x.append(k[6])
        x.append(k[7])
        x.append(k[8])
##years為要預估的年份
        years=5
        year=np.arange(years+1)
        k=np.array(x)
##將比率轉換為百分比存入
        first.append(100*k[0])
        second.append(100*k[1])
        third.append(100*k[2])
##進行5次馬可夫矩陣乘法，每次把遷徙後的分布加入list方便畫圖
        for i in range(1,years+1):
           k=np.dot(A,k)
           first.append(100*k[0])
           second.append(100*k[1])
           third.append(100*k[2])
##繪製直方圖顯示結果，由於3x3矩陣的穩定狀態不好判斷，所以就沒有實作
        width=0.2
        plt.bar(year-0.2,first,width,color='blue',label=name[0])
        plt.bar(year,second,width,color='green',label=name[1])
        plt.bar(year+0.2,third,width,color='yellow',label=name[2])
        plt.set_xticks(year)
        plt.set_xticklabels(name1[0:years+1])
        plt.set_xlabel('Years')
        plt.set_ylabel('Animals Population(%)')
        plt.set_title('Animals Population Predict')
        plt.legend()
        canvas = FigureCanvasTkAgg(f,master=self.threeMatrix)
        canvas.draw()
        canvas.get_tk_widget().place(relx=0.22,rely=0.42)      

if __name__ == "__main__":
  root=PopulationApp()
  root.mainloop()


        

        
        